In [13]:
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
import editdistance

In [2]:

# Load Google's pre-trained Word2Vec model
model = KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True) 
# has plural and upper/lower case, and even bigrams (e.g., taxpayer_dollars; vast_sums)


In [3]:

# flex word2vec's muscles
print "man woman child kitchen*****"
print model.doesnt_match("man woman child kitchen".split())
print "france england germany berlin*****"
print model.doesnt_match("france england germany berlin".split())
print "paris berlin london austria*****"
print model.doesnt_match("paris berlin london austria".split())
print "most similar****"
print model.most_similar("amsterdam")


man woman child kitchen*****
kitchen
france england germany berlin*****
berlin
paris berlin london austria*****
austria
most similar****
[(u'bangkok', 0.5784704685211182), (u'malta', 0.5687170028686523), (u'austria', 0.5659226179122925), (u'switzerland', 0.5658940076828003), (u'deutschland', 0.5655941963195801), (u'gbr', 0.5615140795707703), (u'athens', 0.5609951019287109), (u'las_vegas', 0.5523870587348938), (u'zuma', 0.5522863864898682), (u'london', 0.5517820715904236)]


In [4]:

# Consider a two-person task with a signaler and a receiver (similar to the TV gameshow 'Password'):
# The signalers were told that they would be playing a word-guessing game in which 
# they would have to think of one-word signals that would help someone guess their items. 
# They were talked through an example: if the item was 'dog', then a good signal would be 
# 'puppy' since most people given 'puppy' would probably guess 'dog'.

# sender thinks bank, says money
# receiver think cash
print model.most_similar("bank") # .69 robber, .67 robbery, robbers, security, agency ..
print model.most_similar("money") # .55 dollars, .55 profit, .54 cash
print model.most_similar("cash") # .69 capitalize, .54 money, sell, debt, tax






[(u'banks', 0.7440759539604187), (u'banking', 0.690161406993866), (u'Bank', 0.6698697805404663), (u'lender', 0.6342284679412842), (u'banker', 0.6092953681945801), (u'depositors', 0.6031531691551208), (u'mortgage_lender', 0.5797975659370422), (u'depositor', 0.5716428160667419), (u'BofA', 0.5714625120162964), (u'Citibank', 0.5589520335197449)]
[(u'monies', 0.7165061831474304), (u'funds', 0.7055203914642334), (u'moneys', 0.6289055347442627), (u'dollars', 0.628852367401123), (u'cash', 0.6151220798492432), (u'vast_sums', 0.6057385802268982), (u'fund', 0.5789710283279419), (u'Money', 0.5733489990234375), (u'taxpayer_dollars', 0.5693671107292175), (u'Monies', 0.5586516857147217)]
[(u'money', 0.6151220798492432), (u'Cash', 0.5354235768318176), (u'cows_BCG_matrix', 0.5002978444099426), (u'funds', 0.4898729622364044), (u'marketable_securities', 0.473971962928772), (u'US1bn', 0.46651771664619446), (u'cash_flow', 0.4662739634513855), (u'sweepstakes_winnings', 0.46608874201774597), (u'shortstop_Ped

In [335]:
#print model['money']
print model.similarity("hot","cold") # .20
print model.similarity("hot","warm") # .14


0.4602138689612376
0.4321537079491078


In [332]:
def calculate_edit_distance(word, similar_word_list): 
    len_word = len(word)
    hintword = 'Idontknow'
    for i in similar_word_list:
        ed_dist = editdistance.eval(word.lower(), i.lower())
        if len_word <= 3 and ed_dist <= 2:
            continue
        elif len_word <= 5 and ed_dist < 2:
            continue
        elif 6 <= len_word < 10 and ed_dist <= 4:
            continue
        elif 10 <= len_word and ed_dist <= 6:
            continue
        else:
            if word in i or i in word:
                #print 'word in i'
                continue
            else:
                hintword = i
                #print "hintwordhintwordhintword :  " + hintword
                break
    return hintword

def send_word(password):
    most_similar_word_list = []
    #most_similar_word_list_lower = []
    for i in model.most_similar(password):
        most_similar_word_list.append(i[0])
    #most_similar_word_list.append(password)
    print '########'
  #  for i in most_similar_word_list:
  #      most_similar_word_list_lower.append(i.lower())
  #  print model.doesnt_match(most_similar_word_list_lower)
 #   print '########not lower'
    doestmatch = model.doesnt_match([word.lower() for word in most_similar_word_list])

    most_similar_word_list = [word for word in most_similar_word_list if word.lower() != doestmatch]
    print doestmatch
    print most_similar_word_list
    #most_similar_word_lower = 
    hint = calculate_edit_distance(password, most_similar_word_list)
    return hint

In [333]:
def receive_word(hint):
    most_similar_word_list = []
    for i in model.most_similar(hint):
        most_similar_word_list.append(i[0])
   # print most_similar_word_list
    guessed_password = calculate_edit_distance(hint, most_similar_word_list)
    return str(guessed_password)

In [334]:
test_words = ['cut', 'ice', 'stamp', 'self', 'snail', 'now', 'bed', 'night', 'needle', 'scratch', 'bank', 'joke', 'king', 'salt', 'good', 'washer', 'east', 'nail', 'bulb', 'lost']
for i in test_words:
    hint = send_word(i)
    print 'original_password:   ' + i + '    hint word:   ' +  hint + '     guessed word:   ' +  receive_word(hint)
    
    

########
lop
[u'cutting', u'slash', u'slashed', u'trimmed', u'Cut', u'trimming', u'trim', u'slashing', u'cuts']
original_password:   cut    hint word:   slash     guessed word:   cut
########
melting_polar
[u'Ice', u'Francies_tossed', u'melting_glacial', u'icy', u'Fill_cocktail_shaker', u'caked_oak_tree', u'ice_melts', u'snow', u'cocktail_shaker_filled']
original_password:   ice    hint word:   Francies_tossed     guessed word:   ice
########
stamped
[u'stamps', u'postage_stamps', u'commemorative_stamp', u'semipostal', u'matrimony_affords_Griffin', u'Stamp', u'Telepan_approached', u'commemorative_postage_stamps', u'commemorative_stamps']
original_password:   stamp    hint word:   semipostal     guessed word:   Rare_Audrey_Hepburn
########
hourlong_infomercial
[u'Self', u'narcissistic', u'selfconfidence', u'self_actualization', u'self_righteous', u'self_aggrandizing', u'commandoes_acted', u'narcissism', u'self_loathing']
original_password:   self    hint word:   narcissistic     guessed